In [1]:
import datetime
from ib_async import *
import pandas as pd
from arcticdb import Arctic, QueryBuilder, LibraryOptions

import sys
from pathlib import Path

# Get the parent directory of the current notebook's directory
project_root = Path.cwd().parent

# Add the project root to the system path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db
except:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db

In [2]:
ac = initialize_db('db')


In [3]:
ac.list_libraries()

['general', 'pnl', 'portfolio', 'spx500', 'stocks']

In [4]:
lib = ac.get_library('spx500')
lib.list_symbols()

['ALL_STOCKS']

In [7]:
df = lib.read('ALL_STOCKS').data

In [8]:
df.head()

,Symbol,Open,High,Low,Close,Volume,Name,Sector,Industry,50D_SMA,200D_SMA,ATR,1M,3M,6M,12M,RS IBD,RS Rank,RS Rank 20D MA
Date,,,,,,,,,,,,,,,,,,,
2022-01-06,A,146.810340,147.905131,143.585144,147.076630,2298300.0,Agilent Technologies,Health Care,Life Sciences Tools & Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,EMN,114.486667,115.054309,112.774442,113.667778,1034600.0,Eastman Chemical Company,Materials,Specialty Chemicals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,EMR,91.525911,92.461111,90.915159,91.831284,2477000.0,Emerson Electric,Industrials,Electrical Components & Equipment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,ENPH,156.009995,156.229996,145.820007,151.490005,3546200.0,Enphase,Information Technology,Semiconductor Materials & Equipment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,EOG,86.148351,86.281229,83.774288,84.615837,5254700.0,EOG Resources,Energy,Oil & Gas Exploration & Production,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
import yfinance as yf


In [2]:
yf.download('AMZN',period='1d')

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-08-23,177.339996,178.970001,175.240005,177.039993,177.039993,29095000
